## **How to create a tweet from one of my blog posts using LangChain and Azure OpenAI**

- This notebook shows how you can create the content for a tweet from a given blog post URI.
- It uses LangChain and Azure OpenAi to summarize the entire blog post in a 200 word tweet.

## **What is LangChain?** 

[LangChain](https://python.langchain.com/en/latest/index.html) is a framework for interacting with Large Language Models (LLMs). It provides a helpful abstraction for the most common LLM design patterns:

- Pre-processing the corpus (input data) by iterating over some documents.
- Splitting the documents into chunks.
- Summarizing them.
- Embedding the document chunks in a vector data source.
- When a question is asked, searches the vector data source for similar document chunks.
- Passes these chunks to the LLM, along with the question, to get an answer in response.

### **1. Import dependencies**

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.chains.summarize import load_summarize_chain
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import PromptTemplate

### **2. Configure Azure OpenAI**

In [ ]:
BASE_URL = "https://{BASE}.openai.azure.com"
API_KEY = "PUT-YOUR-API-KEY-HERE"
DEPLOYMENT_NAME = "PUT-YOUR-DEPLOYMENT-NAME-HERE"
llm = AzureChatOpenAI(
    openai_api_base=BASE_URL,
    openai_api_version="2023-03-15-preview",
    deployment_name=DEPLOYMENT_NAME,
    openai_api_key=API_KEY,
    openai_api_type = "azure",
)

### **3. Read the content of a blog post from a given URL**

In [ ]:
url = 'https://www.mytechramblings.com/posts/how-to-integrate-your-roslyn-analyzers-with-sonarqube/'

loader = UnstructuredURLLoader(urls=[
    url
])

data = loader.load()

### **4. Split the entire post into chunks**

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)

texts = text_splitter.split_documents(data)

len(texts)

### **5. Use the LangChain 'map-reduce' chain to geneate the tweet content**

In [ ]:
prompt_template = """
You are the author of the source text.
You need to write a tweet that summarizes the source text.
The tweet must not contain any kind of code.
Make sure the tweet is compelling and well-written. 
The tweet must end with the following phrase: 'More details in: www.mytechramblings.com'
The total tweet size must have no more than 270 characters.

SOURCE:
{text}

TWEET IN ENGLISH:"""

prompt = PromptTemplate(template=prompt_template, input_variables=["text"])

chain = load_summarize_chain(llm, chain_type="map_reduce", combine_prompt=prompt)
chain.run(texts)